# Temporal derivatives

We'll use the feedforward networks from
[Tripp, 2010](http://compneuro.uwaterloo.ca/publications/tripp2010.html).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nengo
import nengo.utils.numpy as npext

from phd.networks.derivatives import FeedforwardDeriv, IntermediateDeriv

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

def test_deriv(deriv_func, dims, ramp=True, t=1.0, dt=0.001, **deriv_args):
    with nengo.Network() as net:
        if ramp:
            timesteps = int(t / dt)
            ramp = np.concatenate([np.linspace(0, 1, timesteps/2), np.linspace(1, 0, timesteps/2)])
            proc = lambda time: ramp[int(time / dt) % ramp.size]
        else:
            proc = nengo.processes.BrownNoise()
        inp = nengo.Node(size_in=dims)
        for i in range(dims):
            nengo.Connection(nengo.Node(output=proc), inp[i])
        ea = nengo.networks.EnsembleArray(n_neurons=40, n_ensembles=dims)
        nengo.Connection(inp, ea.input)
        deriv = deriv_func(dimensions=dims, **deriv_args)
        nengo.Connection(ea.output, deriv.input)
        in_probe = nengo.Probe(inp, synapse=None)
        out_probe = nengo.Probe(deriv.output, synapse=0.01)
    print("%d neurons" % (sum(e.n_neurons for e in net.all_ensembles)))
    sim = nengo.Simulator(net)
    sim.run(t)

    inp = sim.data[in_probe]
    plt.subplot(2, 1, 1)
    plt.plot(sim.trange(), inp)
    plt.ylabel("Input")
    plt.subplot(2, 1, 2)
    plt.plot(sim.trange(), sim.data[out_probe])

### Feedforward, intermediate population

In [ ]:
test_deriv(IntermediateDeriv, n_neurons=50, tau=0.05, dims=2)
test_deriv(IntermediateDeriv, n_neurons=50, tau=0.1, dims=2)

In [ ]:
test_deriv(IntermediateDeriv, n_neurons=50, tau=0.1, dims=2, ramp=False)

### Feedforward, two connections

In [ ]:
test_deriv(FeedforwardDeriv, n_neurons=50, tau_slow=0.05, dims=2)
test_deriv(FeedforwardDeriv, n_neurons=50, tau_slow=0.1, dims=2)

In [ ]:
test_deriv(FeedforwardDeriv, n_neurons=50, tau_slow=0.1, dims=2, ramp=False)